In [1]:
# pip install torch

In [2]:
# !pip install transformers
!pip install datasets

In [3]:
# !pip install --upgrade transformers accelerate

In [4]:
import json, argparse, torch, sys, random, gc, os
import numpy as np
import re
import pandas as pd
import functools
from itertools import chain
from functools import partial
from pathlib import Path
import os
# Transformer
from transformers import (AutoTokenizer, Trainer, TrainingArguments,
                          AutoModelForTokenClassification, DataCollatorForTokenClassification,
                          DebertaV2Config, DebertaV2ForTokenClassification)
from datasets import Dataset, features
from typing import Iterable, Any, Callable
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/ML-Project/PII-DATA'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
train = pd.read_json("/content/drive/My Drive/ML-Project/PII-DATA/train.json")
test= pd.read_json('/content/drive/My Drive/ML-Project/PII-DATA/test.json')

In [7]:
train

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [8]:
train.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [9]:
import ast

# Check the type of the first element in the 'tokens' and 'labels' columns
print(type(train['tokens'].iloc[0]))
print(type(train['labels'].iloc[0]))

# If the types are strings and you need to convert them to lists
if isinstance(train['tokens'].iloc[0], str):
    train['tokens'] = train['tokens'].apply(ast.literal_eval)
if isinstance(train['labels'].iloc[0], str):
    train['labels'] = train['labels'].apply(ast.literal_eval)


<class 'list'>
<class 'list'>


In [10]:
train.head()


,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [11]:

#building up train data and test data
train_data = pd.DataFrame({"doc_id":train["document"],"words":train["tokens"],"labels":train['labels']})

In [12]:
train_data.head()



,doc_id,words,labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,"[Design, Thinking, for, Innovation, \n\n, Sind...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,"[Assignment, :, , Visualization, , Reflecti...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [13]:
import pandas as pd

# Assuming 'train_data' is the original DataFrame you're working with.

# Create a new DataFrame by repeating the doc_id for each token
expanded_doc_id = train_data.explode('words')['doc_id'].reset_index(drop=True)

# Explode the words and labels columns to get a series of tokens and labels
tokens = train_data['words'].explode().reset_index(drop=True)
labels = train_data['labels'].explode().reset_index(drop=True)

# Now create the result DataFrame from the expanded series
result_df = pd.DataFrame({
    'doc_id': expanded_doc_id,
    'token': tokens,
    'label': labels
})

result_df


,doc_id,token,label
0,7,Design,O
1,7,Thinking,O
2,7,for,O
3,7,innovation,O
4,7,reflexion,O
...,...,...,...
4992528,22687,process,O
4992529,22687,explained,O
4992530,22687,above,O
4992531,22687,.,O


In [14]:
# Remove rows where 'token' is whitespace, a newline character, or a dot
result_df = result_df[~result_df['token'].str.match(r'^\s*$|^\\n$|^\.$')]

# If there are any specific strings you consider useless, you can add them to the condition
# For example, to remove rows with tokens that are just 'n\n' you can do the following:
result_df = result_df[result_df['token'] != 'n\n']

# Reset the index after the removal
result_df = result_df.reset_index(drop=True)

result_df

,doc_id,token,label
0,7,Design,O
1,7,Thinking,O
2,7,for,O
3,7,innovation,O
4,7,reflexion,O
...,...,...,...
4434368,22687,of,O
4434369,22687,the,O
4434370,22687,process,O
4434371,22687,explained,O


In [15]:
result_df['label'].value_counts()

O                   4431638
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         16
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: label, dtype: int64

In [16]:
# Strip whitespace from the labels in the DataFrame before mapping
result_df['label'] = result_df['label'].str.strip()

# Recreate the label map in case there are leading/trailing whitespaces in the labels
unique_labels = result_df['label'].unique()
label_map = {label: idx for idx, label in enumerate(unique_labels)}

# Try mapping again
result_df['label'] = result_df['label'].map(label_map)


In [17]:
label_map

{'O': 0,
 'B-NAME_STUDENT': 1,
 'I-NAME_STUDENT': 2,
 'B-URL_PERSONAL': 3,
 'B-EMAIL': 4,
 'B-ID_NUM': 5,
 'I-URL_PERSONAL': 6,
 'B-USERNAME': 7,
 'B-PHONE_NUM': 8,
 'I-PHONE_NUM': 9,
 'B-STREET_ADDRESS': 10,
 'I-STREET_ADDRESS': 11,
 'I-ID_NUM': 12}

In [27]:
result_df['label'].value_counts()

0     4431638
1        1365
2        1096
3         110
5          78
4          39
11         16
9          15
7           6
8           6
10          2
6           1
12          1
Name: label, dtype: int64

In [22]:
from transformers import BertTokenizerFast, BertForTokenClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Group the tokens by 'doc_id' and create a list of lists of tokens
grouped = result_df.groupby('doc_id')['token'].apply(list)
tokens = grouped.tolist()

# Tokenize all tokens at once
encoding = tokenizer.batch_encode_plus(
    tokens,
    is_split_into_words=True,
    add_special_tokens=True,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

# Extract input_ids and attention_masks
input_ids = encoding['input_ids']
attention_masks = encoding['attention_mask']

In [24]:
# Prepare the labels
grouped_labels = result_df.groupby('doc_id')['label'].apply(list)

# Calculate the maximum length after accounting for special tokens
max_length = 128  # this is the padding length you used for tokenization
max_label_length = max_length - 2  # Subtracting the special [CLS] and [SEP] tokens

labels = []
for label_list in grouped_labels:
    # Start with a label for the [CLS] token, followed by the actual labels
    label_array = [-100] + label_list

    # Add a label for the [SEP] token
    label_array += [-100]

    # Add padding if necessary to reach the max_label_length
    label_array += [-100] * (max_label_length - len(label_list))

    # Ensure the label array does not exceed the max_length
    label_array = label_array[:max_length]

    # Append the label array to the list of labels
    labels.append(torch.tensor(label_array))

# Stack the label tensors to create a single tensor
label_ids = torch.stack(labels)




# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
model = BertForTokenClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(set(result_df['label'])),
    output_attentions=False,
    output_hidden_states=False,
)

# Move the model to the device
model.to(device)






Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [25]:



# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)

# Split dataset into train and validation sets
train_size = int(0.8 * len(input_ids))
val_size = len(input_ids) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for train and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Number of epochs to train for
epochs = 4

# Total number of training steps is the number of batches multiplied by the number of epochs
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_loader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f}")

# Save the trained model to the specified directory
model.save_pretrained('/content/drive/My Drive/ML-Project/PII-DATA/model')  # Update this path as needed


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4 | Train Loss: 0.0429
Epoch 2/4 | Train Loss: 0.0048
Epoch 3/4 | Train Loss: 0.0032
Epoch 4/4 | Train Loss: 0.0025


In [26]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Tracking variables for predictions and true labels
val_predictions, val_true_labels = [], []

# Evaluate the model on the validation set
model.eval()  # Set the model to evaluation mode

for batch in validation_loader:
    # Transfer batch to GPU
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Move logits and labels to CPU
    logits = outputs.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Collect the predictions and true labels for each batch
    val_predictions.extend(np.argmax(logits, axis=2).flatten())
    val_true_labels.extend(label_ids.flatten())

# Convert the lists into numpy arrays for evaluation
val_predictions = np.array(val_predictions)
val_true_labels = np.array(val_true_labels)

# Use the true labels and predictions to calculate the classification metrics
print('Validation Accuracy: {:.2f}'.format(accuracy_score(val_true_labels, val_predictions)))

# Use classification_report from sklearn to evaluate the model
report = classification_report(val_true_labels, val_predictions, labels=np.unique(val_true_labels), zero_division=0)
print(report)


Validation Accuracy: 0.99
              precision    recall  f1-score   support

        -100       0.00      0.00      0.00      1547
           0       0.99      1.00      1.00    172481
           1       0.79      0.76      0.78       142
           2       0.75      0.71      0.73       146
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         9
           5       0.00      0.00      0.00         7
           7       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1

    accuracy                           0.99    174336
   macro avg       0.28      0.27      0.28    174336
weighted avg       0.98      0.99      0.99    174336

